# Escopo Geral

A ocorrência de incêndios e queimadas representa um dos mais críticos problemas ambientais enfrentados pelo Brasil, com implicações que vão desde a perda de biodiversidade até impactos severos na saúde humana e no clima global. Apesar da disponibilidade de grandes volumes de dados relacionados a esses eventos, os desafios na comunicação clara e impactante dessas informações permanecem, especialmente quando o público-alvo inclui tanto especialistas quanto a sociedade em geral.

Este projeto busca abordar esses desafios por meio da criação de uma solução inovadora para a representação visual de informações relacionadas a queimadas. Utilizando os dados abertos disponibilizados pela plataforma gov.br. A proposta inicial envolve análises mensais e a progressiva expansão para padrões anuais e sazonais, com foco em destacar tendências, anomalias e áreas críticas.

A solução proposta pelo grupo é a utilização de uma representação iconográfica, com o objetivo de traduzir os dados em elementos gráficos intuitivos, capazes de captar a atenção do público e facilitar a compreensão das informações. A ideia é tornar os dados acessíveis e impactantes, incentivando a conscientização e a ação.

O desenvolvimento da solução será segmentado nas seguintes etapas:
1. Tratamento dos dados: os dados serão baixados e carregados da plataforma (https://terrabrasilis.dpi.inpe.br/queimadas/portal/dados-abertos/#da-focos) e serão verificadas as características dessa base, tendo como foco quais atributos do conjunto serão necessáerios para nosso objetivo. Além disso serão identificados dados faltantes, informações inconsistentes e, também, a necessidade de criação de novos atributos, engenharia de dados etc.;
2. Criação das visualizações: com os dados tratados serão criadas visualizações, cujo intuito é mapear as áreas afetadas por queimadas, onde as características do conjunto serão representadas graficamentepor ícones que representam os atributos do conjunto;
3. Validação dos Resultados: as visualizações criadas serão submetidas para alguns individuos do público geral e de público especialista para verificar a qualidade gráfica da representação das informações

# EXPLORAÇÃO E TRATAMENTO DOS DADOS

A base de dados que será avaliada inicialmente é referente ao mês de novembro de 2024. Inicialmente a etapa de exploração e tratamento será realizada apenas com este conjunto, para obter uma visão inicial de como os dados estão dispostos. A base de dados contém informações detalhadas sobre focos de queimadas no Brasil, organizada em 588.262 registros e 16 colunas. Abaixo estão descritos os atributos presentes:  

### Atributos e Aplicação no Contexto
1. **id (Identificador único)** (`String`):  Identificador único para cada registro, permitindo rastrear individualmente cada foco de queimada.  

2. **lat (Latitude) e lon (Longitude)** (`Float`):  Coordenadas geográficas dos focos de queimadas.  

3. **data_hora_gmt (Data e hora em GMT)** (`String`): Timestamp do momento em que o foco foi registrado.  

4. **satelite (Satélite responsável pelo registro)** (`String`): Indica qual satélite coletou os dados.  

5. **municipio, estado, pais (Localização administrativa)** (`String`): Nome do município, estado e país correspondentes às coordenadas.  

6. **municipio_id, estado_id, pais_id (Identificadores administrativos)** (`Integer`): Códigos numéricos correspondentes às localidades.  

7. **numero_dias_sem_chuva (Número de dias sem chuva)** (`Float`): Indica o tempo seco, um fator relevante para o aumento do risco de queimadas.  

8. **precipitacao (Precipitação em mm)** (`Float`): Quantidade de chuva registrada.  

9. **risco_fogo (Índice de risco de fogo)** (`Float`): Mede a probabilidade de ocorrência de fogo.  

10. **bioma** (`String`): Bioma onde o foco foi registrado (e.g., Amazônia, Cerrado).  

11. **frp (Fire Radiative Power)** (`Float`): Medida da intensidade do fogo, em megawatts.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
from datetime import datetime
from urllib.request import urlopen
import json

In [ ]:
data = pd.read_csv('https://storage.googleapis.com/data-vis-pdssf/focos_mensal_br_202411%20(1).csv', sep=',')

In [ ]:
df = pd.DataFrame(data)

In [ ]:
initial_data_len = len(df['id'])

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588262 entries, 0 to 588261
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     588262 non-null  object 
 1   lat                    588262 non-null  float64
 2   lon                    588262 non-null  float64
 3   data_hora_gmt          588262 non-null  object 
 4   satelite               588262 non-null  object 
 5   municipio              588262 non-null  object 
 6   estado                 588262 non-null  object 
 7   pais                   588262 non-null  object 
 8   municipio_id           588262 non-null  int64  
 9   estado_id              588262 non-null  int64  
 10  pais_id                588262 non-null  int64  
 11  numero_dias_sem_chuva  503812 non-null  float64
 12  precipitacao           503812 non-null  float64
 13  risco_fogo             503812 non-null  float64
 14  bioma                  588262 non-nu

Formatar dados de data e hora

In [ ]:
df['data_hora_gmt'] = pd.to_datetime(df['data_hora_gmt']) # transformar coluna em datetime

date = []
time = []

for i, value in enumerate(df['data_hora_gmt']):
    date.append(value.date().strftime('%y-%m-%d')) # formatar a data e salvar numa lista
    time.append(value.time().strftime('%H:%M:%S')) # formatar a hora e salvar numa lista

df['date'] = date
df['time'] = time

# df.drop(columns=['data_hora_gmt'], inplace=True) # excluir coluna 'data_hora_gmt'

Serão retirados os atributos municipio_id, estado_id, pais_id do conjunto de dados, pois são atributos referentes a um banco de dados relacional que não será utilizado no contexto deste trabalho

In [ ]:
# df.drop(columns=['id', 'pais', 'municipio_id', 'estado_id', 'pais_id'], inplace=True)
df.drop(columns=['id', 'pais', 'pais_id'], inplace=True)

Calculando a porcentagem de valores faltantes

In [ ]:
print("Porcentagem de valores faltantes em:")

for column in df.columns:
    print(f'{column}: {((len(df[column]) - df[column].notna().sum())/df[column].notna().sum())*100} %')

Porcentagem de valores faltantes em:
lat: 0.0 %
lon: 0.0 %
data_hora_gmt: 0.0 %
satelite: 0.0 %
municipio: 0.0 %
estado: 0.0 %
municipio_id: 0.0 %
estado_id: 0.0 %
numero_dias_sem_chuva: 16.762204949465275 %
precipitacao: 16.762204949465275 %
risco_fogo: 16.762204949465275 %
bioma: 0.0 %
frp: 5.524093802861874 %
date: 0.0 %
time: 0.0 %


As colunas com valores faltantes são numero_dias_sem_chuva (16.762%), precipitacao (16.762%), risco_fogo (16.762%), frp (5.524%).

In [ ]:
list_col_miss = ['numero_dias_sem_chuva', 'precipitacao', 'risco_fogo', 'frp']

In [ ]:
for item in list_col_miss:
  index_miss = list(df[item][df[item].isna()].index)
  df.drop(index_miss, inplace=True)

In [ ]:
data_post_treatment_1 = len(df.index)

In [ ]:
print(f'Redução dos dados pós-redução dos valores NaN em: {((initial_data_len - data_post_treatment_1)/initial_data_len)*100} %')

Redução dos dados pós-redução dos valores NaN em: 18.17200499097341 %


Foram identificadas entradas com valor -999

In [ ]:
data_pre_treatment = len(df.index)

In [ ]:
list_col_wrong_value = ['numero_dias_sem_chuva', 'precipitacao', 'risco_fogo', 'frp']

In [ ]:
for item in list_col_wrong_value:
  index_miss = list(df[item][df[item] == -999].index)
  df.drop(index_miss, inplace=True)

In [ ]:
data_post_treatment_2 = len(df.index)

In [ ]:
print(f'Redução dos dados pós-redução dos valores iguais a -999 em: {((data_pre_treatment - data_post_treatment_2)/data_pre_treatment)*100} %')

Redução dos dados pós-redução dos valores iguais a -999 em: 5.068524169909196 %


Redução total dos dados

In [ ]:
print(f'Redução total dos dados em: {((initial_data_len - data_post_treatment_2)/initial_data_len)*100} %')

Redução total dos dados em: 22.319476695758013 %


# Criação das Visualizações

## Visualização Geral

In [ ]:
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    hover_name='municipio',
    hover_data=['date', 'time', 'bioma', 'frp', 'risco_fogo', 'numero_dias_sem_chuva', 'precipitacao'],
    mapbox_style='carto-positron',
    labels={'date': 'Data', 'time': 'Hora', 'bioma': 'Bioma',
            'frp': 'Potência do Fogo', 'risco_fogo': 'Risco de Fogo',
            'lat': 'Latitude', 'lon': 'Longitude', 'precipitacao': 'Precipitação',
            'numero_dias_sem_chuva': 'Dias sem Chuva'},
    zoom=4,
    height=600
)

plotly.offline.plot(fig, filename = f'/content/drive/MyDrive/plots_vis_dados/distribuicao_espacial_brasil.html')

# fig.show()

'/content/drive/MyDrive/plots_vis_dados/distribuicao_espacial_brasil.html'

## Visualizações por Estados

Número de ocorrências de queimadas registradas por estado no mês de novembro.

In [ ]:
estados_dict = {'PARÁ': 'PA',
                'MARANHÃO': 'MA',
                'PIAUÍ': 'PI',
                'AMAPÁ': 'AP',
                'CEARÁ': 'CE',
                'MATO GROSSO': 'MT',
                'AMAZONAS': 'AM',
                'BAHIA': 'BA',
                'PERNAMBUCO': 'PE',
                'TOCANTINS': 'TO',
                'PARAÍBA': 'PB',
                'RORAIMA': 'RR',
                'MINAS GERAIS': 'MG',
                'RIO GRANDE DO NORTE': 'RN',
                'MATO GROSSO DO SUL': 'MS',
                'RONDÔNIA': 'RO',
                'PARANÁ': 'PR',
                'ALAGOAS': 'AL',
                'ACRE': 'AC',
                'SÃO PAULO': 'SP',
                'GOIÁS': 'GO',
                'SERGIPE': 'SE',
                'RIO GRANDE DO SUL': 'RS',
                'SANTA CATARINA': 'SC',
                'ESPÍRITO SANTO': 'ES',
                'RIO DE JANEIRO': 'RJ',
                'DISTRITO FEDERAL': 'DF'}

In [ ]:
df['estado'].replace(estados_dict, inplace=True)

<ipython-input-79-1521b6469f09>:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
queimadas_por_estado = {'estado': list(df['estado'].value_counts().keys()), 'queimadas': list(df['estado'].value_counts().values)}

In [ ]:
df_2 = pd.DataFrame(queimadas_por_estado)

In [ ]:
df_2['codigo_estado'] = df_2['estado'].map({
 'AC': 1, 'AL': 2, 'AM': 3, 'AP': 4, 'BA': 5, 'CE': 6, 'DF': 7,
 'ES': 8, 'GO': 9, 'MA': 10, 'MG': 11, 'MS': 12, 'MT': 13, 'PA': 14,
 'PB': 15, 'PE': 16, 'PI': 17, 'PR': 18, 'RJ': 19, 'RN': 20, 'RO': 21,
 'RR': 22, 'RS': 23, 'SC': 24, 'SE': 25, 'SP': 26, 'TO': 27
})

In [ ]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
  data = json.load(response)

In [ ]:
mes = 'Novembro'
ano = 2024

fig = px.choropleth_mapbox(
      df_2,
      geojson = data,
      locations='codigo_estado',
      featureidkey = 'properties.id',
      color = 'queimadas',
      # animation_frame = 'date',
      hover_name = 'estado',
      hover_data = ['queimadas'],
      title = f'Distribuição de Queimadas no Brasil em {mes} de {ano}',
      color_continuous_scale='YlOrRd',
      mapbox_style = 'carto-positron',
      center = {'lat': -15.83, 'lon': -47.86},
      labels={'queimadas': 'Queimadas'},
      zoom = 2.5,
      opacity = 0.9)

fig.update_geos(fitbounds = 'locations', visible = False)
plotly.offline.plot(fig, filename = f'/content/drive/MyDrive/plots_vis_dados/map_queimadas_brasil_{mes}_{ano}.html')

# fig.show()

'/content/drive/MyDrive/plots_vis_dados/map_queimadas_brasil_Novembro_2024.html'

## Visualizações por Biomas

Podemos ainda identificar no mapa do Brasil, onde estão os focos de incêndio de cada bioma, quanto maior a área colorida, maior o número de focos ou mais potente são as queimadas no local.

No gráfico é possível ainda identificar a cidade onde se deu a queimada e a potência do fogo.

In [ ]:
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    color='bioma',
    size='frp',
    hover_name='municipio',
    title='Distribuição Espacial por Bioma',
    mapbox_style='carto-positron',
    labels={'bioma': 'Bioma', 'frp': 'Potência do Fogo', 'lat': 'Latitude', 'lon': 'Longitude'},
    zoom=4,
    height=600
)

plotly.offline.plot(fig, filename = f'/content/drive/MyDrive/plots_vis_dados/distribuicao_espacial_por_bioma.html')

# fig.show()

'/content/drive/MyDrive/plots_vis_dados/distribuicao_espacial_por_bioma.html'

## Visualizações por Municípios

In [ ]:
centro_estados = {
	'AC': [-8.77, -70.55], 'AL': [-9.62, -36.82], 'AM': [-3.47, -65.10], 'AP': [1.41, -51.77],
	'BA': [-13.29, -41.71], 'CE': [-5.20, -39.53], 'DF': [-15.83, -47.86], 'ES': [-19.19, -40.34],
	'GO': [-15.98, -49.86], 'MA': [-5.42, -45.44], 'MT': [-12.64, -55.42], 'MS': [-20.51, -54.54],
	'MG': [-18.10, -44.38], 'PA': [-3.79, -52.48], 'PB': [-7.28, -36.72], 'PR': [-24.89, -51.55],
	'PI': [-6.60, -42.28], 'RJ': [-22.25, -42.66], 'RN': [-5.81, -36.59], 'RO': [-10.83, -63.34],
	'RR': [1.99, -61.33], 'SC': [-27.45, -50.95], 'SE': [-10.57, -37.45], 'SP': [-22.19, -48.79],
  'PE': [-8.38, -37.86], 'RS': [-30.17, -53.50],	'TO': [-9.46, -48.26]
}

In [ ]:
estado_id = {}

for i in range(27):
  estado_id[data['features'][i]['properties']['sigla']] = data['features'][i]['properties']['codigo_ibg']

In [ ]:
for estado in estados_dict:
  df_3 = df[df['estado'] == estados_dict[estado]]

  mun = dict(zip(df_3['municipio_id'], df_3['municipio']))

  df_4 = pd.DataFrame({'nome': [mun[i] for i in df_3['municipio_id'].value_counts().keys()], 'municipio': df_3['municipio_id'].value_counts().keys(), 'queimadas': df_3['municipio_id'].value_counts().values})

  with urlopen(f'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-{estado_id[estados_dict[estado]]}-mun.json') as response:
      geo_json = json.load(response)

  fig = px.choropleth_mapbox(
      df_4,
      geojson = geo_json,
      locations='municipio',
      featureidkey = 'properties.id',
      color = 'queimadas',
      # animation_frame = 'date',
      hover_name = 'nome',
      hover_data = 'queimadas',
      title = f'Queimadas em {estado}',
      color_continuous_scale='YlOrRd',
      mapbox_style = 'carto-positron',
      center = {'lat': centro_estados[estados_dict[estado]][0], 'lon': centro_estados[estados_dict[estado]][1]},
      zoom = 3,
      labels={'queimadas': 'Queimadas'},
      opacity = 0.9)

  fig.update_geos(fitbounds = 'locations', visible = False)
  # plotly.offline.plot(fig, filename = f'/content/drive/MyDrive/plots_vis_dados/por_estado/{estados_dict[estado]}_map_queimadas.html')
  # fig.show()

# Classificação dos Dados

Classificação dos dados por Fire Power

In [ ]:
q1, q2, q3 = df['frp'].quantile([0.25, 0.5, 0.75])
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df['cat_frp'] = pd.cut(df['frp'],
                       bins=[0, q1, q2, q3, upper_bound, df['frp'].max()],
                       labels=['Muito Baixa', 'Baixa', 'Media', 'Alta', 'Muito Alta'])

Classificação dos dados por risco de fogo

In [ ]:
q1, q2, q3 = df['risco_fogo'].quantile([0.25, 0.5, 0.75])

df['cat_risco_fogo'] = pd.cut(df['risco_fogo'],
                              bins=[0, q1, q2, q3],
                              labels=['Baixo', 'Medio', 'Alto'])

Criação das Visualizações classificadas

In [ ]:
_ = {'frp': 'Potência do Fogo', 'risco_fogo': 'Risco de Fogo'}

for metrica in ['frp', 'risco_fogo']:
  for categoria in ['Baixo', 'Medio', 'Alto'] if metrica == 'risco_fogo' else ['Muito Baixa', 'Baixa', 'Media', 'Alta', 'Muito Alta']:
    fig = px.scatter_mapbox(
        df[df[f'cat_{metrica}'] == categoria],
        lat='lat',
        lon='lon',
        color=f'cat_{metrica}',
        hover_name='municipio',
        hover_data=['date', 'time', 'bioma', 'frp', 'risco_fogo'],
        title=f'Distribuição Espacial das Queimadas de {_[metrica]} {categoria}s',
        mapbox_style='carto-positron',
        zoom=2.5,
        labels={'date': 'Data', 'time': 'Hora', 'bioma': 'Bioma', 'frp': 'Potência do Fogo',
                'risco_fogo': 'Risco de Fogo', 'lat': 'Latitude', 'lon': 'Longitude',
                'cat_frp': 'Potência do Fogo', 'cat_risco_fogo': 'Risco de Fogo'},
        height=600
    )
    plotly.offline.plot(fig, filename = f'/content/drive/MyDrive/plots_vis_dados/por_categoria/{metrica}/queimadas_{categoria}_{metrica}.html')
    # fig.show()